In [1]:
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data,DataLoader
from functions import *
from pytorch_util import *
from torch.optim import Adam
from torch.nn.utils import clip_grad_value_

In [2]:
with open('../Data/train_data_ACSF.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data_ACSF.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data_ACSF.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [3]:
### parameters ###
batch_size = 32
dim = 64
edge_dim = 12
epochs = 5
clip = 2

### load dataset

In [4]:
train_list = [Data(**d) for d in train_data]
train_dl = DataLoader(train_list,batch_size,shuffle=True)

In [5]:
val_list = [Data(**d) for d in val_data]
valid_dl = DataLoader(val_list,batch_size,shuffle=False)

In [6]:
test_list = [Data(**d) for d in test_data]
test_dl = DataLoader(test_list,batch_size,shuffle=False)

In [63]:
data = next(iter(train_dl))

In [40]:
class Net_int_2Edges_intHead(torch.nn.Module):
    # use both types of edges
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges_intHead, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        self.lin_edge_attr = torch.nn.Linear(edge_in, edge_dim)
        
        nn1 = Linear(edge_dim, dim * dim, bias=False)
        nn2 = Linear(edge_in3, dim * dim * 2 * 2, bias=False)
        
        self.conv1 = NNConv(dim, dim, nn1, aggr='mean', root_weight=False)
        self.gru1 = GRU(dim, dim)
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = NNConv(dim*2, dim*2, nn2, aggr='mean', root_weight=False)
        self.gru2 = GRU(dim*2, dim*2)
        
        self.head = InteractionNet2(edge_in3,[dim*3*2,dim*2,1],[F.relu,None])
        
        self.norm = BatchNorm1d(dim*3*2)
        self.norm_x = BatchNorm1d(node_in)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        edge_attr = F.rrelu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        edge_attr3 = torch.cat([data.edge_attr3,data.edge_attr3],0)
        
        for i in range(2):
            # using bonding as edge
            m = F.rrelu(self.conv1(out, data.edge_index, edge_attr))
            out, h = self.gru1(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        for i in range(2):
            # using couping as edge
            m = F.rrelu(self.conv2(out, edge_index3, edge_attr3))
            out, h = self.gru2(m.unsqueeze(0), h)
            out = out.squeeze(0)     
            
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        yhat = self.head(data.edge_attr3,yhat)
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat 

In [48]:
class Net_int_2Edges_pool_intHead(torch.nn.Module):
    # use set2set for global pooling and cat results with local features
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges_pool_intHead, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        self.lin_edge_attr = torch.nn.Linear(edge_in, edge_dim)
        
        nn1 = Linear(edge_dim, dim * dim, bias=False)
        nn2 = Linear(edge_in3, dim * dim * 2 * 2, bias=False)
        
        self.conv1 = NNConv(dim, dim, nn1, aggr='mean', root_weight=False)
        self.gru1 = GRU(dim, dim)
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = NNConv(dim*2, dim*2, nn2, aggr='mean', root_weight=False)
        self.gru2 = GRU(dim*2, dim*2)
        
        self.head = InteractionNet2(edge_in3,[dim*5*2,dim*2,1],[F.relu,None])
        
        self.norm = BatchNorm1d(dim*5*2)
        self.norm_x = BatchNorm1d(node_in)
        self.set2set = Set2Set(dim*2,processing_steps=3)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        edge_attr = F.rrelu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        edge_attr3 = torch.cat([data.edge_attr3,data.edge_attr3],0)
        
        for i in range(2):
            # using bonding as edge
            m = F.rrelu(self.conv1(out, data.edge_index, edge_attr))
            out, h = self.gru1(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        for i in range(2):
            # using couping as edge
            m = F.rrelu(self.conv2(out, edge_index3, edge_attr3))
            out, h = self.gru2(m.unsqueeze(0), h)
            out = out.squeeze(0)  
            
        coupling_batch_index = data.batch[data.edge_index3[0]]
        pool = self.set2set(out, data.batch) # (m,d)
        pool = pool[coupling_batch_index] # (n_target,d)
        temp = out[data.edge_index3] # (2,n_target,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2,pool],1)  
        
        yhat = self.norm(yhat)
        yhat = self.head(data.edge_attr3,yhat)
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat  

In [87]:
class Net_int_2Edges_pool_intHead2(torch.nn.Module):
    # use set2set for global pooling and cat results with local features
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges_pool_intHead2, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        self.lin_edge_attr = torch.nn.Linear(edge_in, edge_dim)
        
        nn1 = Linear(edge_dim, dim * dim, bias=False)
        nn2 = Linear(edge_in3, dim * dim * 2 * 2, bias=False)
        
        self.conv1 = NNConv(dim, dim, nn1, aggr='mean', root_weight=False)
        self.gru1 = GRU(dim, dim)
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = NNConv(dim*2, dim*2, nn2, aggr='mean', root_weight=False)
        self.gru2 = GRU(dim*2, dim*2)
        
        self.head = InteractionNet2(edge_in3+dim*4,[dim*3*2,dim,1],[F.relu,None])
        
        self.norm = BatchNorm1d(dim*3*2)
        self.norm_x = BatchNorm1d(node_in)
        self.set2set = Set2Set(dim*2,processing_steps=3)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        edge_attr = F.rrelu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        edge_attr3 = torch.cat([data.edge_attr3,data.edge_attr3],0)
        
        for i in range(2):
            # using bonding as edge
            m = F.rrelu(self.conv1(out, data.edge_index, edge_attr))
            out, h = self.gru1(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        for i in range(2):
            # using couping as edge
            m = F.rrelu(self.conv2(out, edge_index3, edge_attr3))
            out, h = self.gru2(m.unsqueeze(0), h)
            out = out.squeeze(0)  
            
        coupling_batch_index = data.batch[data.edge_index3[0]]
        pool = self.set2set(out, data.batch) # (m,d)
        pool = pool[coupling_batch_index] # (n_target,d)
        temp = out[data.edge_index3] # (2,n_target,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)  
        
        yhat = self.norm(yhat)
        yhat = self.head(torch.cat([data.edge_attr3,pool],1),yhat)
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat  

### build model and set up training

In [88]:
#model = Net_int_2Edges_intHead(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')
#model = Net_int_2Edges_pool_intHead(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')
model = Net_int_2Edges_pool_intHead2(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')

In [89]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [90]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.7873947438198408, val_loss:0.3341339266516714
epoch:1, train_loss:0.36166528322119446, val_loss:0.24919676350859496
epoch:2, train_loss:0.22757536750312368, val_loss:0.10041901867231752
epoch:3, train_loss:0.1381506137720215, val_loss:-0.02990294844469326
epoch:4, train_loss:0.0665987360708085, val_loss:0.04686835084237859
Training completed in 575.0818157196045s


In [91]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.00524437175761307, val_loss:-0.11224827195048077
epoch:1, train_loss:-0.04776853368888857, val_loss:-0.13469644623179722
epoch:2, train_loss:-0.09216798324637016, val_loss:-0.19477214771681106
epoch:3, train_loss:-0.13292237297279627, val_loss:-0.24870299455574435
epoch:4, train_loss:-0.16989954037035662, val_loss:-0.29617804620001054
Training completed in 645.3577108383179s


In [93]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.20147026187489445, val_loss:-0.3301054004293222
epoch:1, train_loss:-0.231122358219641, val_loss:-0.35216580302669453
epoch:2, train_loss:-0.25693018601250855, val_loss:-0.3782724665525632
epoch:3, train_loss:-0.2834316911282308, val_loss:-0.4049747976928185
epoch:4, train_loss:-0.3059644651010285, val_loss:-0.43008813596306705
Training completed in 661.7355451583862s


In [94]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.32840665769995375, val_loss:-0.4471948258897178
epoch:1, train_loss:-0.34614883554366066, val_loss:-0.47605571195355845
epoch:2, train_loss:-0.3652804598355326, val_loss:-0.48447955495271927
epoch:3, train_loss:-0.3845258105697837, val_loss:-0.4832447755158457
epoch:4, train_loss:-0.4010357999629935, val_loss:-0.5053740528404204
Training completed in 749.0423629283905s


In [96]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.4195739028998363, val_loss:-0.5370073982665682
epoch:1, train_loss:-0.43429612570366033, val_loss:-0.5289927729301982
epoch:2, train_loss:-0.445725753521832, val_loss:-0.5260609407455493
epoch:3, train_loss:-0.46146462580413106, val_loss:-0.5786263877127924
epoch:4, train_loss:-0.475618605144666, val_loss:-0.5662320554893241
Training completed in 928.5973472595215s


In [98]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.48830208976109607, val_loss:-0.5802539597846504
epoch:1, train_loss:-0.49995962164148, val_loss:-0.6050142347175851
epoch:2, train_loss:-0.5147427366471585, val_loss:-0.5544515141309836
epoch:3, train_loss:-0.528825285816825, val_loss:-0.5788627008342335
epoch:4, train_loss:-0.5388606865492159, val_loss:-0.6163132952956053
Training completed in 932.0772387981415s


In [101]:
for para in opt.param_groups:
    para['lr'] = 5e-5

In [102]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.594168065520362, val_loss:-0.6928425587267957
epoch:1, train_loss:-0.6068908522049477, val_loss:-0.6865779711013167
epoch:2, train_loss:-0.6141801948519019, val_loss:-0.7026917973899434
epoch:3, train_loss:-0.6220309867208059, val_loss:-0.7057264423650554
epoch:4, train_loss:-0.6282204711442676, val_loss:-0.7012094151642587
Training completed in 927.6188719272614s


In [103]:
for para in opt.param_groups:
    para['lr'] = 2e-5

In [104]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.6633511964431025, val_loss:-0.7569363331183409
epoch:1, train_loss:-0.6710359787335939, val_loss:-0.7478445477337918
epoch:2, train_loss:-0.6752695218171494, val_loss:-0.7589000694007955
epoch:3, train_loss:-0.6774823888148355, val_loss:-0.7486525864427925
epoch:4, train_loss:-0.6821317726011193, val_loss:-0.7608554126360477
Training completed in 932.4170694351196s


In [106]:
# Net_int_2Edges_pool_intHead2
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.6843306843308918, val_loss:-0.7611134335016593
epoch:1, train_loss:-0.6876784494929408, val_loss:-0.7651663442325388
epoch:2, train_loss:-0.6915706446114118, val_loss:-0.7709255159920098
epoch:3, train_loss:-0.6920657717287677, val_loss:-0.7760558013732617
epoch:4, train_loss:-0.6943480713513543, val_loss:-0.7782296613495574
Training completed in 931.6694602966309s


In [51]:
# Net_int_2Edges_pool_intHead
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.0586407478944755, val_loss:0.5972323164853275
epoch:1, train_loss:0.5822940412781039, val_loss:0.3499437818924586
epoch:2, train_loss:0.4129772612669177, val_loss:0.2494932159733696
epoch:3, train_loss:0.31060964641406974, val_loss:0.14766907464298937
epoch:4, train_loss:0.23812104661526967, val_loss:0.09794970082007667
Training completed in 1035.262710094452s


In [43]:
# Net_int_2Edges_intHead
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.9432164053421997, val_loss:0.4776822461022271
epoch:1, train_loss:0.4486014823364132, val_loss:0.2770513335927429
epoch:2, train_loss:0.30067624731196296, val_loss:0.15217620228281897
epoch:3, train_loss:0.21372147340789116, val_loss:0.06608518575214678
epoch:4, train_loss:0.14415263022697702, val_loss:0.00784918664691922
Training completed in 644.1707198619843s


In [45]:
# checkpoint = torch.load('../Model/gnn_int_logloss_2Edges.tar')
# model.load_state_dict(checkpoint['model_state_dict'])
# opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [107]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, '../Model/Net_int_2Edges_pool_intHead2.tar')

### make submissions

In [108]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/Net_int_2Edges_pool_intHead2.csv',index=False)